#notebook setup

In [1]:
!pip install -qq tf-nightly-gpu-2.0-preview

     |████████████████████████████████| 395.5MB 21kB/s 
     |████████████████████████████████| 3.8MB 2.7MB/s 
     |████████████████████████████████| 450kB 6.5MB/s 
     |████████████████████████████████| 81kB 2.6MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
ERROR: tb-nightly 2.1.0a20191206 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.


In [2]:
!pip freeze

absl-py==0.8.1
alabaster==0.7.12
albumentations==0.1.12
altair==4.0.0
asgiref==3.2.3
astor==0.8.1
astropy==3.0.5
atari-py==0.2.6
atomicwrites==1.3.0
attrs==19.3.0
audioread==2.1.8
autograd==1.3
Babel==2.7.0
backcall==0.1.0
backports.tempfile==1.0
backports.weakref==1.0.post1
beautifulsoup4==4.6.3
bleach==3.1.0
blis==0.2.4
bokeh==1.0.4
boto==2.49.0
boto3==1.10.40
botocore==1.13.40
Bottleneck==1.3.1
branca==0.3.1
bs4==0.0.1
bz2file==0.98
cachetools==4.0.0
certifi==2019.11.28
cffi==1.13.2
chainer==6.5.0
chardet==3.0.4
chart-studio==1.0.0
Click==7.0
cloudpickle==1.2.2
cmake==3.12.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.2.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.0
cupy-cuda101==6.5.0
cvxopt==1.2.3
cvxpy==1.0.25
cycler==0.10.0
cymem==2.0.3
Cython==0.29.14
daft==0.0.4
dask==1.1.5
dataclasses==0.7
datascience==0.10.6
decorator==4.4.1
defusedxml==0.6.0
descartes==1.1.0
dill==0.3.1.1
distributed==1.25.3
Django==3.0
dlib==19.18.0
dm-sonnet==1.3

In [3]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import sys
try:
    import ujson as json
except ImportError:
    try:
        import simplejson as json
    except ImportError:
        import json

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from sklearn.preprocessing import MinMaxScaler
import pywt
import os
from PIL import Image
# import tensorflow as tf
from glob import glob

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Model,load_model,Sequential

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout,Flatten, Input
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Data prep

In [6]:
%cd drive/'My Drive/lhc_durham'
!pwd

/content/drive/My Drive/lhc_durham
/content/drive/My Drive/lhc_durham


In [7]:
img_folder = '/filtered_images'
!ls 'filtered_images'

0  1  2


class 0 : DYjets
class 1 : TTjets
class 2 : Wjets
class 3 : ZZjets

## Train val size setting

In [8]:
cwd = os.getcwd()
img_path = cwd + img_folder

train_val_size = 1200

df = {}
data = []
for idx, x in enumerate(os.listdir(img_path)):
  if x.isdigit() and idx < 3:
    for idx2, pic in enumerate(os.listdir(img_path+'/'+str(x))):
      if idx2 < train_val_size:
        data.append((img_path+ '/' + x + '/' + pic , x))

df = pd.DataFrame(data, columns=['filename', 'class'])
df['class'] = df['class'].astype(int)
#shuffle
df = df.sample(frac=1).reset_index(drop=True)

print(df.head())
print(len(df))
df.dtypes

                                            filename  class
0  /content/drive/My Drive/lhc_durham/filtered_im...      2
1  /content/drive/My Drive/lhc_durham/filtered_im...      2
2  /content/drive/My Drive/lhc_durham/filtered_im...      0
3  /content/drive/My Drive/lhc_durham/filtered_im...      0
4  /content/drive/My Drive/lhc_durham/filtered_im...      1
3600


filename    object
class        int64
dtype: object

In [9]:
from collections import Counter
cnt = Counter(df['class'])
cnt

Counter({0: 1200, 1: 1200, 2: 1200})

In [10]:
# Split into train and validation sets
train_percentage = .7
val_percentage = 0.15
test_percentage = 0.15

train_length = int(len(df) * train_percentage)
df_train = df[:train_length]

val_length = int(len(df) * val_percentage)
df_val = df[train_length : train_length+val_length]

test_length = int(len(df) * test_percentage)
df_test = df[ train_length + val_length : ]

# shuffle 
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_val = df_val.sample(frac=1).reset_index(drop=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)

df_list = [df_train, df_val, df_test]
for df_part in df_list:
  print(len(df_part))

2520
540
540


#### image parsing

In [0]:
# Reads an image from a file, decodes it into a tensor, and resizes it
# to a fixed shape.
img_rows, img_cols = 200,200
num_classes = len(set(df['class']))
batch_size = 256

def _parse_function(filename, label):
  image_string = tf.io.read_file(filename)
  image_decoded = tf.image.decode_jpeg(image_string)
  image_resized = tf.image.resize(image_decoded, [img_rows, img_cols])
  image_resized = tf.ensure_shape(image_resized ,shape=(img_rows, img_cols,3))
  label = tf.one_hot(label, num_classes)
  return image_resized, label

In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(df_train['filename'].values, tf.string),
                                                    tf.cast(df_train['class'].values, tf.int32) ))
train_dataset = train_dataset.map(_parse_function)
train_dataset = train_dataset.shuffle(5000)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.batch(batch_size, drop_remainder=True)

In [0]:
valid_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(df_val['filename'].values, tf.string),
                                                    tf.cast(df_val['class'].values, tf.int32) ))
valid_dataset = valid_dataset.map(_parse_function)
valid_dataset = valid_dataset.shuffle(5000)
valid_dataset = valid_dataset.repeat()
valid_dataset = valid_dataset.batch(batch_size, drop_remainder=True)

In [0]:
test_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(df_test['filename'].values, tf.string),
                                                    tf.cast(df_test['class'].values, tf.int32) ))
test_dataset = test_dataset.map(_parse_function)
test_dataset = test_dataset.shuffle(5000)
test_dataset = test_dataset.repeat()
test_dataset = test_dataset.batch(batch_size, drop_remainder=True)

In [15]:
train_dataset

<BatchDataset shapes: ((256, 200, 200, 3), (256, 3)), types: (tf.float32, tf.float32)>

# inception

In [20]:
base_inception = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(200, 200, 3), pooling=None)

87916544/87910968 [==============================] - 2s 0us/step


In [0]:
# add a global spatial average pooling layer
x = base_inception.output

x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)

x = Dropout(0.5)(x)

#x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)

x = Dropout(0.5)(x)

# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)

x = Dropout(0.5)(x)

# and a logits layer -- let's say we have 3 classes
predictions = Dense(3, activation='softmax')(x)

In [0]:
inception_model = Model(inputs=base_inception.input, outputs=predictions)

In [23]:
for layer in base_inception.layers:
    layer.trainable = False
    print(layer.name, ' Trainable =', layer.trainable)
    

input_1  Trainable = False
conv2d  Trainable = False
batch_normalization  Trainable = False
activation  Trainable = False
conv2d_1  Trainable = False
batch_normalization_1  Trainable = False
activation_1  Trainable = False
conv2d_2  Trainable = False
batch_normalization_2  Trainable = False
activation_2  Trainable = False
max_pooling2d  Trainable = False
conv2d_3  Trainable = False
batch_normalization_3  Trainable = False
activation_3  Trainable = False
conv2d_4  Trainable = False
batch_normalization_4  Trainable = False
activation_4  Trainable = False
max_pooling2d_1  Trainable = False
conv2d_8  Trainable = False
batch_normalization_8  Trainable = False
activation_8  Trainable = False
conv2d_6  Trainable = False
conv2d_9  Trainable = False
batch_normalization_6  Trainable = False
batch_normalization_9  Trainable = False
activation_6  Trainable = False
activation_9  Trainable = False
average_pooling2d  Trainable = False
conv2d_5  Trainable = False
conv2d_7  Trainable = False
conv2d_10 

In [24]:
for layer in inception_model.layers:
    print(layer.name,' Trainable =',layer.trainable)

input_1  Trainable = False
conv2d  Trainable = False
batch_normalization  Trainable = False
activation  Trainable = False
conv2d_1  Trainable = False
batch_normalization_1  Trainable = False
activation_1  Trainable = False
conv2d_2  Trainable = False
batch_normalization_2  Trainable = False
activation_2  Trainable = False
max_pooling2d  Trainable = False
conv2d_3  Trainable = False
batch_normalization_3  Trainable = False
activation_3  Trainable = False
conv2d_4  Trainable = False
batch_normalization_4  Trainable = False
activation_4  Trainable = False
max_pooling2d_1  Trainable = False
conv2d_8  Trainable = False
batch_normalization_8  Trainable = False
activation_8  Trainable = False
conv2d_6  Trainable = False
conv2d_9  Trainable = False
batch_normalization_6  Trainable = False
batch_normalization_9  Trainable = False
activation_6  Trainable = False
activation_9  Trainable = False
average_pooling2d  Trainable = False
conv2d_5  Trainable = False
conv2d_7  Trainable = False
conv2d_10 

In [0]:
#opt = tf.keras.optimizers.Adam(lr=0.001)
opt = tf.keras.optimizers.RMSprop(lr=1e-4)

# compile the model (should be done *after* setting layers to non-trainable)
inception_model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

In [26]:
train_steps = int(len(df)/batch_size) #total trains set / batch_size
val_steps = int(len(df_val)/batch_size)


print('train steps:',train_steps)
print('val steps:',val_steps)

train steps: 14
val steps: 2


In [27]:
epochs = 10

### Train the model with validation 
inception_history = inception_model.fit( train_dataset, steps_per_epoch = train_steps,
                   epochs = epochs,
                   validation_data = valid_dataset,
                   validation_steps = val_steps)

Train for 14 steps, validate for 2 steps
Epoch 1/10
14/14 [==============================] - 1209s 86s/step - loss: 1.1718 - accuracy: 0.3624 - val_loss: 15.5247 - val_accuracy: 0.3398
Epoch 2/10
14/14 [==============================] - 9s 678ms/step - loss: 1.0676 - accuracy: 0.4308 - val_loss: 18.6768 - val_accuracy: 0.3535
Epoch 3/10
14/14 [==============================] - 13s 964ms/step - loss: 1.0066 - accuracy: 0.4914 - val_loss: 23.5423 - val_accuracy: 0.3301
Epoch 4/10
14/14 [==============================] - 9s 668ms/step - loss: 0.9283 - accuracy: 0.5544 - val_loss: 25.3122 - val_accuracy: 0.3242
Epoch 5/10
14/14 [==============================] - 14s 972ms/step - loss: 0.8825 - accuracy: 0.5801 - val_loss: 37.0807 - val_accuracy: 0.3379
Epoch 6/10
14/14 [==============================] - 9s 664ms/step - loss: 0.8256 - accuracy: 0.6116 - val_loss: 38.1529 - val_accuracy: 0.3320
Epoch 7/10
14/14 [==============================] - 9s 668ms/step - loss: 0.7803 - accuracy: 0.633

In [0]:
inception_model.trainable = True

set_trainable = False
for layer in inception_model.layers:
    if layer.name == 'conv2d_181':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [50]:
for layer in inception_model.layers:
    print(layer.name,' Trainable =',layer.trainable)

input_3  Trainable = False
conv2d  Trainable = False
batch_normalization  Trainable = False
activation  Trainable = False
conv2d_1  Trainable = False
batch_normalization_1  Trainable = False
activation_1  Trainable = False
conv2d_2  Trainable = False
batch_normalization_2  Trainable = False
activation_2  Trainable = False
max_pooling2d  Trainable = False
conv2d_3  Trainable = False
batch_normalization_3  Trainable = False
activation_3  Trainable = False
conv2d_4  Trainable = False
batch_normalization_4  Trainable = False
activation_4  Trainable = False
max_pooling2d_1  Trainable = False
conv2d_8  Trainable = False
batch_normalization_8  Trainable = False
activation_8  Trainable = False
conv2d_6  Trainable = False
conv2d_9  Trainable = False
batch_normalization_6  Trainable = False
batch_normalization_9  Trainable = False
activation_6  Trainable = False
activation_9  Trainable = False
average_pooling2d  Trainable = False
conv2d_5  Trainable = False
conv2d_7  Trainable = False
conv2d_10 

In [51]:
epochs = 50

	
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=50)

# Train the model with validation 
tuned_inception_history = inception_model.fit( train_dataset, steps_per_epoch = train_steps,
                   epochs = epochs,
                   validation_data = valid_dataset,
                   validation_steps = val_steps,
                   callbacks=[es])

Train for 17 steps, validate for 2 steps
Epoch 1/50
17/17 [==============================] - 18s 1s/step - loss: 0.3618 - accuracy: 0.8463 - val_loss: 37.8424 - val_accuracy: 0.3184
Epoch 2/50
17/17 [==============================] - 12s 722ms/step - loss: 0.3589 - accuracy: 0.8421 - val_loss: 49.6048 - val_accuracy: 0.3320
Epoch 3/50
17/17 [==============================] - 18s 1s/step - loss: 0.3792 - accuracy: 0.8304 - val_loss: 59.1348 - val_accuracy: 0.3457
Epoch 4/50
17/17 [==============================] - 12s 725ms/step - loss: 0.3308 - accuracy: 0.8591 - val_loss: 57.4081 - val_accuracy: 0.3301
Epoch 5/50
17/17 [==============================] - 12s 716ms/step - loss: 0.3058 - accuracy: 0.8697 - val_loss: 110.4535 - val_accuracy: 0.3516
Epoch 6/50
17/17 [==============================] - 18s 1s/step - loss: 0.2873 - accuracy: 0.8780 - val_loss: 84.0913 - val_accuracy: 0.3477
Epoch 7/50
17/17 [==============================] - 12s 715ms/step - loss: 0.2744 - accuracy: 0.8860 - 

KeyboardInterrupt: ignored

# vgg

## basic

In [28]:
base_vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3),pooling=None)

58892288/58889256 [==============================] - 2s 0us/step


In [0]:
# add a global spatial average pooling layer
x = base_vgg.output

x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)

x = Dropout(0.3)(x)

# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)

x = Dropout(0.3)(x)

# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)

x = Dropout(0.3)(x)

# and a logits layer -- let's say we have 3 classes
predictions = Dense(3, activation='softmax')(x)

In [0]:
# this is the model we will train
base_vgg_model = Model(inputs=base_vgg.input, outputs=predictions)

In [31]:
for layer in base_vgg.layers:
    print(layer.name)
    layer.trainable = False

input_2
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [32]:
for layer in base_vgg_model.layers:
    print(layer.name,' Trainable =',layer.trainable)

input_2  Trainable = False
block1_conv1  Trainable = False
block1_conv2  Trainable = False
block1_pool  Trainable = False
block2_conv1  Trainable = False
block2_conv2  Trainable = False
block2_pool  Trainable = False
block3_conv1  Trainable = False
block3_conv2  Trainable = False
block3_conv3  Trainable = False
block3_pool  Trainable = False
block4_conv1  Trainable = False
block4_conv2  Trainable = False
block4_conv3  Trainable = False
block4_pool  Trainable = False
block5_conv1  Trainable = False
block5_conv2  Trainable = False
block5_conv3  Trainable = False
block5_pool  Trainable = False
global_average_pooling2d_1  Trainable = True
dense_4  Trainable = True
dropout_3  Trainable = True
dense_5  Trainable = True
dropout_4  Trainable = True
dense_6  Trainable = True
dropout_5  Trainable = True
dense_7  Trainable = True


In [0]:
opt = tf.keras.optimizers.RMSprop(lr=1e-4)

# compile the model (should be done *after* setting layers to non-trainable)
base_vgg_model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

In [34]:
train_steps = int(len(df)/batch_size) #total trains set / batch_size
val_steps = int(len(df_val)/batch_size)
#epochs = 1

print('train steps:',train_steps)
print('val steps:',val_steps)
epochs = 50

### Train the model with validation 
vgg_history = base_vgg_model.fit( train_dataset, steps_per_epoch = train_steps,
                   epochs = epochs,
                   validation_data = valid_dataset,
                   validation_steps = val_steps)

train steps: 14
val steps: 2
Train for 14 steps, validate for 2 steps
Epoch 1/200
14/14 [==============================] - 21s 2s/step - loss: 1.5742 - accuracy: 0.4445 - val_loss: 0.6975 - val_accuracy: 0.6387
Epoch 2/200
14/14 [==============================] - 13s 963ms/step - loss: 1.0931 - accuracy: 0.5586 - val_loss: 0.6370 - val_accuracy: 0.7129
Epoch 3/200
14/14 [==============================] - 17s 1s/step - loss: 0.9563 - accuracy: 0.5968 - val_loss: 0.6120 - val_accuracy: 0.7148
Epoch 4/200
14/14 [==============================] - 13s 952ms/step - loss: 0.8370 - accuracy: 0.6300 - val_loss: 0.6110 - val_accuracy: 0.7383
Epoch 5/200
14/14 [==============================] - 17s 1s/step - loss: 0.7937 - accuracy: 0.6431 - val_loss: 0.6275 - val_accuracy: 0.7266
Epoch 6/200
14/14 [==============================] - 13s 949ms/step - loss: 0.7272 - accuracy: 0.6694 - val_loss: 0.6056 - val_accuracy: 0.7500
Epoch 7/200
14/14 [==============================] - 14s 968ms/step - loss:

In [1]:
test_steps = int(len(df_test)/batch_size)

metrics = base_vgg_model.evaluate(test_dataset,
                   steps = test_steps)
print("model accuracy:",metrics[1])

NameError: ignored

## 1st unfreezing

In [0]:
base_vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3),pooling=None)

In [0]:
x = base_vgg.output

x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)

x = Dropout(0.3)(x)

# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)

x = Dropout(0.3)(x)

# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)

x = Dropout(0.3)(x)

# and a logits layer -- let's say we have 3 classes
predictions = Dense(3, activation='softmax')(x)

In [0]:
base_vgg_model.trainable = True

set_trainable = False
for layer in base_vgg_model.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [29]:
for layer in base_vgg_model.layers:
    print(layer.name,layer.trainable)

input_1 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_pool False
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True
global_average_pooling2d True
dense True
dropout True
dense_1 True


#### fitting the model

In [30]:


epochs = 50

	
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=15)

vgg_model_filepath = 'small_unfrozen_vgg16.h5'
mcp = ModelCheckpoint(vgg_model_filepath, monitor='accuracy', save_best_only=True, mode='max')

# Train the model with validation 
tuned_vgg_history = base_vgg_model.fit( train_dataset, steps_per_epoch = train_steps,
                   epochs = epochs,
                   validation_data = valid_dataset,
                   validation_steps = val_steps,
                   callbacks=[es, mcp])

Train for 17 steps, validate for 2 steps
Epoch 1/200
17/17 [==============================] - 24s 1s/step - loss: 0.4264 - accuracy: 0.8118 - val_loss: 0.4985 - val_accuracy: 0.7871
Epoch 2/200
17/17 [==============================] - 18s 1s/step - loss: 0.3903 - accuracy: 0.8295 - val_loss: 0.5119 - val_accuracy: 0.7832
Epoch 3/200
17/17 [==============================] - 23s 1s/step - loss: 0.3957 - accuracy: 0.8284 - val_loss: 0.4664 - val_accuracy: 0.8027
Epoch 4/200
17/17 [==============================] - 18s 1s/step - loss: 0.3788 - accuracy: 0.8366 - val_loss: 0.5116 - val_accuracy: 0.7812
Epoch 5/200
17/17 [==============================] - 17s 1s/step - loss: 0.3772 - accuracy: 0.8339 - val_loss: 0.4974 - val_accuracy: 0.7812
Epoch 6/200
17/17 [==============================] - 24s 1s/step - loss: 0.3631 - accuracy: 0.8470 - val_loss: 0.4850 - val_accuracy: 0.7773
Epoch 7/200
17/17 [==============================] - 18s 1s/step - loss: 0.3543 - accuracy: 0.8502 - val_loss: 0.

KeyboardInterrupt: ignored

In [0]:
base_vgg_model.save(cwd + '/models/vgg_tuned.h5')

In [0]:
best_vgg_model = load_model(cwd + '/models/vgg_tuned.h5')

#### saving the history of the fitting

In [0]:
import pickle

with open(cwd + '/trainHistoryDict/vgg_history.p', 'wb') as file_pi:
        pickle.dump(tuned_vgg_history.history, file_pi)

In [31]:
from tensorflow.keras.models import load_model

best_vgg_model = load_model(vgg_model_filepath)

In [32]:
test_steps = int(len(df_test)/batch_size)

metrics = best_vgg_model.evaluate(test_dataset,
                   steps = test_steps)
print("model accuracy:",metrics[1])

metrics = base_vgg_model.evaluate(test_dataset,
                   steps = test_steps)
print("model accuracy:",metrics[1])

2/2 [==============================] - 3s 1s/step - loss: 0.8058 - accuracy: 0.7754
model accuracy: 0.7753906
2/2 [==============================] - 2s 1s/step - loss: 0.8219 - accuracy: 0.7852
model accuracy: 0.78515625


In [0]:
11584/15000

0.7722666666666667

## entirely unfrozen

In [16]:
base_vgg = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3),pooling=None)

58892288/58889256 [==============================] - 1s 0us/step


In [0]:
# add a global spatial average pooling layer
x = base_vgg.output

x = GlobalAveragePooling2D()(x)

# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)

x = Dropout(0.3)(x)

# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)

x = Dropout(0.3)(x)

# let's add a fully-connected layer
x = Dense(512, activation='relu')(x)

x = Dropout(0.3)(x)

# and a logits layer -- let's say we have 3 classes
predictions = Dense(3, activation='softmax')(x)

In [0]:
# this is the model we will train
unfrozen_vgg_model = Model(inputs=base_vgg.input, outputs=predictions)

In [21]:
for layer in unfrozen_vgg_model.layers:
    print(layer.name)
    layer.trainable = True

input_1
block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool
global_average_pooling2d
dense
dropout
dense_1
dropout_1
dense_2
dropout_2
dense_3


In [22]:
for layer in unfrozen_vgg_model.layers:
    print(layer.name,' Trainable =',layer.trainable)

input_1  Trainable = True
block1_conv1  Trainable = True
block1_conv2  Trainable = True
block1_pool  Trainable = True
block2_conv1  Trainable = True
block2_conv2  Trainable = True
block2_pool  Trainable = True
block3_conv1  Trainable = True
block3_conv2  Trainable = True
block3_conv3  Trainable = True
block3_pool  Trainable = True
block4_conv1  Trainable = True
block4_conv2  Trainable = True
block4_conv3  Trainable = True
block4_pool  Trainable = True
block5_conv1  Trainable = True
block5_conv2  Trainable = True
block5_conv3  Trainable = True
block5_pool  Trainable = True
global_average_pooling2d  Trainable = True
dense  Trainable = True
dropout  Trainable = True
dense_1  Trainable = True
dropout_1  Trainable = True
dense_2  Trainable = True
dropout_2  Trainable = True
dense_3  Trainable = True


#### setting the optimizer

In [0]:
opt = tf.keras.optimizers.Adam(lr=0.001)

# compile the model (should be done *after* setting layers to non-trainable)
unfrozen_vgg_model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])

In [25]:
train_steps = int(len(df)/batch_size) #total trains set / batch_size
val_steps = int(len(df_val)/batch_size)

print('train steps:',train_steps)
print('val steps:',val_steps)

train steps: 14
val steps: 2


#### fitting the model

In [26]:
epochs = 50

	
es = EarlyStopping(monitor='accuracy', mode='max', verbose=1, patience=50)

vgg_model_filepath = 'unfrozen_vgg16.h5'
mcp = ModelCheckpoint(vgg_model_filepath, monitor='accuracy', save_best_only=True, mode='max')

# Train the model with validation 
unfrozen_vgg_history = unfrozen_vgg_model.fit( train_dataset, steps_per_epoch = train_steps,
                   epochs = epochs,
                   validation_data = valid_dataset,
                   validation_steps = val_steps,
                   callbacks=[es, mcp])

Train for 14 steps, validate for 2 steps
Epoch 1/50
14/14 [==============================] - 1113s 80s/step - loss: 2.2100 - accuracy: 0.3401 - val_loss: 1.0983 - val_accuracy: 0.3809
Epoch 2/50
14/14 [==============================] - 29s 2s/step - loss: 1.1155 - accuracy: 0.3373 - val_loss: 1.0982 - val_accuracy: 0.3457
Epoch 3/50
14/14 [==============================] - 36s 3s/step - loss: 1.2143 - accuracy: 0.3451 - val_loss: 1.0985 - val_accuracy: 0.3418
Epoch 4/50
14/14 [==============================] - 29s 2s/step - loss: 1.1688 - accuracy: 0.3295 - val_loss: 1.0986 - val_accuracy: 0.3398
Epoch 5/50
14/14 [==============================] - 33s 2s/step - loss: 1.0988 - accuracy: 0.3304 - val_loss: 1.0987 - val_accuracy: 0.3340
Epoch 6/50
14/14 [==============================] - 29s 2s/step - loss: 1.0985 - accuracy: 0.3329 - val_loss: 1.0986 - val_accuracy: 0.3281
Epoch 7/50
14/14 [==============================] - 29s 2s/step - loss: 1.0984 - accuracy: 0.3410 - val_loss: 1.0985

KeyboardInterrupt: ignored

#### saving the history of the fitting

In [0]:
import pickle

with open(cwd + '/trainHistoryDict/vgg_history.p', 'wb') as file_pi:
        pickle.dump(tuned_vgg_history.history, file_pi)